In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import system assys
warnings.filterwarnings("ignore")
import math
import scipy.stats as st
from sklearn import preprocessing

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_log_error, make_scorer, mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold, RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import pickle

# plt.style.use('ggplot')
# plt.rcParams['figure.figsize'] = (18,12)

## Make Scorers

In [ ]:
def rmsle(y, y_hat):
    res = 0
    for i, j in zip(y, y_hat):
        res += (math.log(i + 1) - math.log(j + 1)) ** 2
    return (res / len(y)) ** 0.5

def rmse(y_actual, y_pred):
    return np.sqrt(mean_squared_error(y_actual, y_pred))

rmsle_score = make_scorer(rmsle, greater_is_better=False)
rmse_score = make_scorer(rmse, greater_is_better=False)

## Data Uploading

In [ ]:
# объединенный трейн и тест датасеты с геогр.координатами
with open('df.pkl', 'rb') as f:
    df_coordinates = pickle.load(f)
f.close()
del f

In [ ]:
df_train = df_coordinates[df_coordinates['sampletype']=='train']
df_train.shape

In [ ]:
df_test = df_coordinates[df_coordinates['sampletype']=='test']
df_test.shape

In [ ]:
del df_coordinates

## Test

## EDA

In [ ]:
def df_print_unique(df_input):
    df_output = df_input.fillna(-100)
    for col in df_output.columns:
        print(df_output[col].value_counts())
        print(f'nunique = {df_output[col].nunique()}')
        print("~"*50)
        
#df_print_unique(df_train)

In [ ]:
# for col in df_train.columns:
#     print(df_train.groupby('Рубрика')[col].count())
#     print('---------------------------------')

## Filling NaN

In [ ]:
# for col in df_train.columns:
#     print(col)
#     print(pd.unique(df_train[col].values))
#     print('---------------------------------')

In [ ]:
# кол-во пропусков в процентах
#df_train.isna().sum()*100/df_train.shape[0]

In [ ]:
def fill_nan(df_input):
    df_output = df_input.copy()
    fill_with_minus_1 = ['Коммунальные платежи включены',
                  'Находится в залоге',
                  'Внесен задаток',
                  'Возможен обмен',
                  'Ипотека',
                  'Чистая продажа',
                  'Торг',
                  'Интернет',
                  'Наличие мебели',
                  'Наличие холодильника',
                  'Площадь кухни',
                  'Жилая площадь',
                  'Общая площадь',
                  'Год сдачи',
                  'Санузел',
                  'Ремонт',
    ]
    
    fill_with_other = ['Ориентир',
                      'Тип квартиры',
                      'Планировка',
                      'Форма собственности',
                      'Тип дома',       
                      'Материал дома'                     
    ]
    
    fill_with_unkn = ['Улица',
                      'Номер дома',
                      'Район', 
                      'Микрорайон'
    ]
    
    df_output[fill_with_minus_1] = df_output[fill_with_minus_1].fillna(value=-1)
    df_output[fill_with_other] = df_output[fill_with_other].fillna(value='другое')
    df_output[fill_with_unkn] = df_output[fill_with_unkn].fillna(value='не указано')
    
    return df_output

## Data Cleaning (preprocessing outliers + garbage)

In [ ]:
def clean_test_data(df_input):
    df_output = df_input.copy()

    df_output['Форма собственности'] = df_output['Форма собственности'].replace(
        {'другое (укажите в описании)': 'другое'}) 
    df_output['Материал дома'] = df_output['Материал дома'].replace({'кирпич-монолит':'кирпич - монолит'})
    
    # исправляем выборосы из координат: сравниваем реальные координаты городов и регионов с раскодированными, удаляем
    # слишком далекие друг от друга (по евклидовому расстоянию), заменяем на средние lon и lat по городу
#     df_tmp = pd.merge(
#                 df_output, df_output.groupby(['Регион', 'Город']).agg({'lat': 'mean', 'lon': 'mean'}).reset_index(), 
#                 how='left', 
#                 left_on=['Регион', 'Город'], 
#                 right_on=['Регион', 'Город'])
#     df_tmp['dist'] = df_tmp.apply(
#         lambda r: ((r['lat_x'] - r['lat_y'])**2 + (r['lon_x'] - r['lon_y'])**2)**0.5, axis=1)
#     for i in df_tmp[(~df_tmp.location_full_match) & (df_tmp.dist > 5)].index.values:
#         if str(df_tmp.loc[i, 'Регион']).lower() not in str(df_tmp.loc[i, 'location_display_name']).lower() \
#         and str(df_tmp.loc[i, 'Город']).lower() not in str(df_tmp.loc[i, 'location_display_name']).lower():
#             df_output.loc[i, 'lat'] = df_tmp.loc[i, 'lat_y']
#             df_output.loc[i, 'lon'] = df_tmp.loc[i, 'lon_y']
            
            
    # удаляем выборосы из всей выборки
    outliers = [] 
    
    outliers.extend(df_output[(df_output['Цена']>=10000000)&(df_output['Общая площадь']<=50)|
                              (df_output['Цена']>20000000)&(df_output['Общая площадь']<100)|
                              (df_output['Цена']==350000000)|
                              (df_output['Цена']==111111111)|
                              (df_output['Цена']==85000000)|
                              (df_output['Цена']==68000000)|
                              (df_output['Общая площадь']<=5)
                             ].index)
    outliers.extend(df_output[(df_output['Этажность']>52)].index)
    outliers.extend(df_output[(df_output['Количество лоджий']>20)|(df_output['Количество балконов']>20)].index)
    outliers.extend(
        df_output[
            (df_output['Общая площадь']>400)|
            (df_output['Площадь кухни']>df_output['Общая площадь'])|
            (df_output['Жилая площадь']>df_output['Общая площадь'])].index
    )
    
    df_output = df_output.drop(df_output.index[outliers])
    print('Удалено общих выбросов:', len(outliers))
    
#     # ДЛЯ АРЕНДЫ
#     outliers_rent = []
#     outliers_rent.extend(
#         (df_output[(df_output['Рубрика']=='Аренда')&(
#             (df_output['Цена']<1000)|
#             (df_output['Цена']>150000))]).index)
#     df_output = df_output.drop(outliers_rent)
#     print('Удалено выбросов аренды:', len(outliers_rent))
    
#     # ДЛЯ ПРОДАЖИ
#     outliers_sale = []
#     outliers_sale.extend(
#         (df_output[(df_output['Рубрика']=='Продажа')&(
#             (df_output['Цена']<50000)|
#             (df_output['Цена']>45000000))]).index)
#     df_output = df_output.drop(outliers_sale)
#     print('Удалено выбросов продаж:', len(outliers_sale))
    
    return df_output

In [ ]:
# plt.scatter(df_train['lat'], df_train['lon'])
# plt.scatter(df_train_preproc_NA_OUT['lat'], df_train_preproc_NA_OUT['lon'])
# plt.ylabel('lon')
# plt.xlabel('lat')

## Feature Encoding

In [ ]:
# for c in df_train_preproc_NA_OUT.columns:
#     if df_train_preproc_NA_OUT[c].dtype == 'object':
#         print(df_train_preproc_NA_OUT.groupby(c)[c].count())
#         print('----------------------------')

In [ ]:
def feature_encoding(df_input):
    df_output = df_input.copy()
    
    features_to_ohe = [] # порядок на переменных не задан
    
    df_output['Количество комнат'] = df_output['Количество комнат'].replace({'комната': 0.5}).astype('float64')
    
    df_output['Ремонт'] = df_output['Ремонт'].replace(
                                                        {
                                                        'требует капитального ремонта':1,
                                                        'требует косметического ремонта':10,
                                                        'хорошее':20,
                                                        'в отличном состоянии':100},
    ).astype('int64') 
    
    df_output['Санузел'] = df_output['Санузел'].replace(
                                                        {'другое':1,
                                                        'совмещенный':2,
                                                        'раздельный':3,
                                                        'несколько':6}
    ).astype('int64') 
    
    df_output['Планировка'] = df_output['Планировка'].replace(
                                                        {'другое': -1,
                                                        'смежная':1,
                                                        'смежно-изолированная':10,
                                                        'свободная':20,
                                                        'изолированная':100},
    ).astype('int64') 
    
    df_output['Тип квартиры'] = df_output['Тип квартиры'].replace(
                                                        {'другое': -1,
                                                        'гостинка':1,
                                                        'студия':2,
                                                        'улучшенной планировки':3,
                                                        'индивидуальной планировки':4,
                                                        'двухуровневая':5,
                                                        'пентхаус':6}
     ).astype('int64')
        
    df_output['Материал дома'] = df_output['Материал дома'].replace(
                                                        {'другое': -1,
                                                        'сборный железобетон':1,
                                                        'панель':2,
                                                        'монолит':3,
                                                        'кирпич - монолит':4,
                                                        'кирпич':5,
                                                        'шлакоблоки':6,
                                                        'силикатные блоки':7,
                                                        'бетонные блоки':8,
                                                        'бетонные блоки - монолит':9,
                                                        'дерево':9}
     ).astype('int64')
    
    
    df_output['Форма собственности'] = df_output['Форма собственности'].replace(
                                                        {'другое': -1,
                                                        'не оформлена':1,
                                                        'государственная':2,
                                                        'предварительный договор':3,
                                                        'участие в ЖСК':4,
                                                        'кооперативная':5,
                                                        'инвестиционная':6,
                                                        'договор долевого участия':7,
                                                        'частная':8,
                                                        'свидетельство о праве собственности':9}
     ).astype('int64')
    
    df_output['Тип дома'] = df_output['Тип дома'].replace(
                                                        {'другое': -1,
                                                        'общежитие':1,
                                                        'секционного типа':2,
                                                        'малоэтажка':3,
                                                        'ульяновка':4,
                                                        'хрущевка':5,
                                                        'брежневка':6,
                                                        'сталинка':7,
                                                        'ленинградский проект':8,
                                                        '93 серия':9,
                                                        '95 серия':10,
                                                        '97 серия':11,
                                                        '97 серия улучшенная':12,
                                                        '121 серия':13,
                                                        '121Т':14,
                                                        'спецпроект':15}
     ).astype('int64')
       
    return df_output

In [ ]:
df_train_preproc_NA_OUT_ENC = df_train.pipe(fill_nan).pipe(clean_test_data).pipe(feature_encoding)

In [ ]:
# def fill_na_with_predict (df_input):
#     df_output = df_input.copy()
    
#     # заполняем пропуски Общей площади методом ближайших соседей
#     knn = KNeighborsRegressor(n_neighbors=15, weights='distance')
#     knn.fit(df_output.loc[df_output['Общая площадь'].notna(), ['lat', 'lon', 'Количество комнат']], 
#             df_output.loc[df_output['Общая площадь'].notna(), 'Общая площадь'])
#     df_output.loc[df_output['Общая площадь'].isna(), 'Общая площадь'] = \
#     knn.predict(df_output.loc[df_output['Общая площадь'].isna(), ['lat', 'lon', 'Количество комнат']])
    
#     return df_output

#df_train_preproc_NA_OUT_ENC = df_train_preproc_NA_OUT_ENC.pipe(fill_na_with_predict)

In [ ]:
#df_print_unique(df_train_preproc_NA_OUT_ENC)

In [ ]:
# for c in df_train_preproc_NA_OUT_ENC.columns:
#     if df_train_preproc_NA_OUT_ENC[c].dtype == 'object':
#         print(df_train_preproc_NA_OUT.groupby(c)[c].count())
#         print('----------------------------')

In [ ]:
def check_skewness_log(col, train):
    sns.distplot(np.log1p(train[col]+1), fit=st.norm);
    fig = plt.figure()
    (mu, sigma) = st.norm.fit(train[col])

In [ ]:
def check_skewness(col, train):
    sns.distplot(train[col]+1, fit=st.norm);
    fig = plt.figure()
    (mu, sigma) = st.norm.fit(train[col])

In [ ]:
# for c in df_train_preproc_NA_OUT_ENC.columns:
#     if df_train_preproc_NA_OUT_ENC[c].dtype != 'object':
#         check_skewness_log(c, df_train_preproc_NA_OUT_ENC)

## Visualization

In [ ]:
# plt.figure(figsize=(20,10))
# sns.heatmap(df_train_preproc_NA_OUT_ENC.corr('kendall'), annot=True, linewidths=.5)

In [ ]:
# # most correlated features
# corrmat = df_train_preproc_NA_OUT_ENC.corr()
# top_corr_features = corrmat.index[abs(corrmat["Цена"])>0.3]
# plt.figure(figsize=(7,5))
# g = sns.heatmap(df_train_preproc_NA_OUT_ENC[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [ ]:
# sns.pairplot(df_train_rent, vars=['Количество комнат', 'Общая площадь', 'Жилая площадь', 'Площадь кухни',
#        'Этаж', 'Этажность','Цена'], diag_kind='hist')

In [ ]:
# sns.pairplot(df_train_rent, vars=['lat_h', 'lon_h', 'lat_centr',
#        'lon_centr', 'dist_center', 'dist_center2', 'dist_center3',
#        'dist_center4', 'house_age', 'house_age2', 'index_price_perv',
#        'index_price_vtor', 'inds', 'zp', 'zp2', 'metro', 'Цена'], diag_kind='hist')

## Feature engineering  

In [ ]:
import re

In [ ]:
def add_features(df_input):
    df_output = df_input.copy()

    centroid = df_output[['Город', 'lat', 'lon']].groupby('Город').mean()
    df_output = df_output.merge(centroid, on='Город', suffixes=('_h', '_centr'), right_index=True)
    df_output['dist_center'] = ((df_output.lat_h-df_output.lat_centr)**2+(df_output.lon_h-df_output.lon_centr)**2)**0.5
    df_output['dist_center2'] = np.log1p(df_output['dist_center'])
    df_output['house_age'] = df_output['Год сдачи'].apply(lambda x: -1 if x<0 else 2019-x if 2019-x>0 else 0.5)
    df_output['not_first_floor'] = df_output['Этаж'].apply(lambda x: 0 if x==1 else -1 if x==-1 else 1)
    df_output['tmp'] = df_output['Этаж']/df_output['Этажность']
    df_output['not_top_floor'] = df_output['tmp'].apply(lambda x: 0 if x==1 else -1 if x<0 else 1)
    df_output['has_balcony'] = (df_output['Количество балконов'].apply(lambda x: 0 if x<=0 else 1) + \
                                    df_output['Количество лоджий'].apply(lambda x: 0 if x<=0 else 1))/2
    df_output['index_price_perv'] = df_output['Регион'].replace(
                                                        {
                                                        'Новосибирская область':105.7,
                                                        'Свердловская область':103.3,
                                                        'Омская область':102.9,
                                                        'Тюменская область':103.0,
                                                        'Челябинская область':101.0,
                                                        'Красноярский край':102.1,
                                                        'Пермский край':106.4,
                                                        'Архангельская область':100.7,
                                                        }
                            )
    df_output['index_price_vtor'] = df_output['Регион'].replace(
                                                        {
                                                        'Новосибирская область':101.8,
                                                        'Свердловская область':103.0,
                                                        'Омская область':105.1,
                                                        'Тюменская область':100.3,
                                                        'Челябинская область':100.1,
                                                        'Красноярский край':101.2,
                                                        'Пермский край':92.4,
                                                        'Архангельская область':101.4,
                                                        }
                            )
    df_output['zp'] = df_output['Регион'].replace(
                                                        {
                                                        'Новосибирская область':37173,
                                                        'Свердловская область':36853,
                                                        'Омская область':34136,
                                                        'Тюменская область':72289,
                                                        'Челябинская область':36619,
                                                        'Красноярский край':48980,
                                                        'Пермский край':37173,
                                                        'Архангельская область':51236,
                                                        }
                            )
    df_output['metro'] = df_output['Ориентир'].apply(lambda x: 1 if re.search('метро', str(x)) else -1 )
    enc = preprocessing.LabelEncoder()
    df_output['town'] = enc.fit_transform(df_output['Город'])
    
    #df_output.plot(x='metro', y='Цена', kind='scatter')
    
    cols_to_drop = [
        'Регион',
        'Город',
        'Улица',
        'Номер дома',
        'Район',
        'Микрорайон',
        'Ориентир',
        'Рубрика', 
        'sampletype',
        'address',
        'location_display_name',
        'location_full_match',
    ]
    
    df_output = df_output.drop(cols_to_drop, axis=1)
    
    return df_output

## Modeling

In [ ]:
# Добавляем новые фичи, делим тестовую выбрку на аренду и продажу, отделяем таргет (цену)
df_train_rent = df_train_preproc_NA_OUT_ENC[df_train_preproc_NA_OUT_ENC['Рубрика'] == 'Аренда'].pipe(add_features)
df_train_sale = df_train_preproc_NA_OUT_ENC[df_train_preproc_NA_OUT_ENC['Рубрика'] == 'Продажа'].pipe(add_features)

cols_to_drop_rent = [
        'Цена', 'Торг', 'Чистая продажа', 'Ипотека', 'lat_centr', 'lon_centr',
        'Возможен обмен', 'Внесен задаток', 'Находится в залоге', 'tmp', 'town',
        'index_price_perv', 'index_price_vtor', 'zp', 'dist_center2'
]

cols_to_drop_sale = [
       'Цена', 'Наличие мебели', 'Наличие холодильника', 'Коммунальные платежи включены', 
       'lat_centr', 'lon_centr', 'tmp', 'index_price_perv', 'index_price_vtor', 'zp', 'dist_center2'
]

df_train_rent_target = df_train_rent['Цена']
df_train_sale_target = df_train_sale['Цена']

df_train_rent.drop(cols_to_drop_rent, axis=1, inplace=True)
df_train_sale.drop(cols_to_drop_sale, axis=1, inplace=True)

del df_train_preproc_NA_OUT_ENC
del df_train

In [ ]:
print('Аренда:',df_train_rent.columns,'\n-----\n',df_train_rent.shape,df_train_rent_target.shape,'\n-----')
print('Продажа:',df_train_sale.columns,'\n-----\n',df_train_sale.shape,df_train_sale_target.shape,'\n-----')

## Rent

In [ ]:
def rent_model_01(X_train, X_test, y_train):
        class RentKNNImputer(BaseEstimator, TransformerMixin):
            def fit(self, X, y):
                return self
            def transform(self, X):
                cols = [
                    'Количество комнат', 
                    'Общая площадь', 
                    'Площадь кухни', 
                    'lat_h', 
                    'lon_h',
                    'Жилая площадь',
                    'dist_center',
                    'dist_center2',
                    'dist_center3',
                    'dist_center4',
                    'house_age'
            ]
                return X[cols].copy()

        pipe = Pipeline([
            ('imputer', RentKNNImputer()),
            ('scaler', StandardScaler()),
            ('knn', KNeighborsRegressor()),
        ])

        param_grid = {
            'knn__n_neighbors': [12],
            'knn__p': [1]
        }
        
        kfold = KFold(n_splits=3, shuffle=True, random_state=0)
        hyper_search = GridSearchCV(pipe, param_grid, scoring=rmse_score, cv=kfold, n_jobs=2, refit=True, verbose=2)
        hyper_search.fit(X_train, y_train)
        model = hyper_search.best_estimator_
        print(f"RentKNNModel: best_params = {hyper_search.best_params_}, best_score = {hyper_search.best_score_}")
        y_pred = model.predict(X_test)
        
        return y_pred

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_train_rent, df_train_rent_target, random_state=0, test_size=0.5)
X_train.shape, X_test.shape, y_train.shape, y_test.shape
y_train = np.log1p(y_train)
y_test = np.log1p(y_test)

In [ ]:
pred_knn = rent_model_01(X_train, X_test, y_train)
pred_knn

In [ ]:
print(f"score = {rmsle(np.expm1(y_test), np.expm1(pred_knn))}") #score = 0.23534122382702258

In [ ]:
pred_knn.shape

In [ ]:
X_test_knn = X_test.copy()
X_test_knn = pd.concat([X_test_knn, pd.Series(pred_knn, index=X_test.index, name='knn')], axis=1, sort=False)

In [ ]:
X_test_knn['knn'] = np.log1p(X_test_knn['knn'])
y_test_knn = y_test.copy()
#y_test_knn = np.log1p(y_test)

In [ ]:
def rent_model_02(X_train, X_test, y_train):
        class RentLRImputer(BaseEstimator, TransformerMixin):
            def fit(self, X, y):
                return self

            def transform(self, X):
                cols = [
                    'Количество комнат', 
                    'Общая площадь', 
                    'Жилая площадь', 
                    'Площадь кухни',
                    'Этаж', 
                    'Этажность', 
                    'Тип квартиры', 
                    'Планировка', 
                    'Ремонт', 
                    'Санузел',
                    'Количество балконов', 
                    'Количество лоджий', 
                    'Интернет', 
                    'Торг',
                    'Чистая продажа', 
                    'Ипотека', 
                    'Возможен обмен', 
                    'Внесен задаток',
                    'Находится в залоге', 
                    'Форма собственности', 
                    'Год сдачи', 
                    'Тип дома',
                    'Материал дома', 
                    'Количество фотографий', 
                    'lat_h', 
                    'lon_h', 
                    'dist_center', 
                    'dist_center2',
#                     'dist_center3',
#                     'dist_center4',
                    'index_price_perv',
                    'index_price_vtor',
                    #'inds',
                    'zp',
                    'metro',
                    'house_age',
                    'not_first_floor',
                    'not_top_floor',
                    'has_balcony',
                    #'region',
                    #'town',
                    #'urb'
                ]
                return X[cols].copy()

        pipeline = Pipeline([
            ('imputer', RentLRImputer()),
            ('scaler', MinMaxScaler()),
            ('lr', Ridge()),
        ])

        param_grid = {
            'lr__alpha': [1, 10, 100, 1000],
        }
        
        hyper_search = GridSearchCV(pipeline, param_grid, scoring=rmse_score, cv=3, n_jobs=5, refit=True, verbose=2)
        hyper_search.fit(X_train, y_train)
        model = hyper_search.best_estimator_
        print(f"RentLRModel:best_params = {hyper_search.best_params_}, best_score = {hyper_search.best_score_}")
        y_pred = model.predict(X_test)
        return y_pred

In [ ]:
#X_train2, X_test2, y_train2, y_test2 = train_test_split(X_test_knn, y_test_knn, random_state=0, test_size=0.3)
#y2 = np.log1p(y_test_knn)

In [ ]:
pred_lr = rent_model_02(X_train2, X_test2, y_train2)

In [ ]:
print(f"score = {rmsle(np.expm1(y_test2), np.expm1(pred_lr))}") #score = 0.4293694740000044

In [ ]:
def rent_model_03(X_train, X_test, y_train):
        class RentRFRImputer(BaseEstimator, TransformerMixin):
            def fit(self, X, y):
                return self
            def transform(self, X):
                cols = [
                    'Количество комнат', 
                    'Общая площадь', 
                    'Площадь кухни', 
                    'Этаж',
                    'Этажность', 
                    'Тип квартиры', 
                    'Планировка', 
                    'Ремонт', 
                    'Санузел',
                    'Количество балконов', 
                    'Количество лоджий', 
                    'Наличие мебели',
                    'Наличие холодильника', 
                    'Интернет',
                    'Коммунальные платежи включены', 
                    'Тип дома', 
                    'Материал дома',
                    'Количество фотографий', 
                    'lat_h', 
                    'lon_h',
                    'Год сдачи',
                    'Жилая площадь',
                    'dist_center',
                    'dist_center2',
                    'dist_center3',
                    'dist_center4',
                    'index_price_perv',
                    'index_price_vtor',
                     'inds',
                     'zp',
#                     'zp2',
                    'metro',
                    'house_age',
                    #'not_first_not_top',
                    'not_first_floor',
                    'not_top_floor',
                    'has_balcony',
                    'region',
                    'town',
                    #'urb'
            ]
                return X[cols].copy()

        pipe = Pipeline([
            ('imputer', RentRFRImputer()),
            ('rfr', RandomForestRegressor()),
        ])

        param_grid={
            'rfr__max_depth': [25],
            'rfr__n_estimators': [10],
            #'rfr__max_features': ['sqrt'],
            #'rfr__min_samples_split': range(2, 10),
            #'rfr__min_samples_leaf': range(1, 500, 50),
            #'rfr__warm_start': [True],
            #'rfr__bootstrap': [False],
        },
        
        kfold = KFold(n_splits=3, shuffle=True, random_state=0)
        hyper_search = GridSearchCV(pipe, param_grid, scoring=rmsle_score, cv=kfold, n_jobs=2, refit=True, verbose=50)
        hyper_search.fit(X_train, y_train)
        model = hyper_search.best_estimator_
        print(f"RentRFRModel: best_params = {hyper_search.best_params_}, best_score = {hyper_search.best_score_}")
        
#         # посмотрим на важность признаков
#         features = X_train.columns
#         importances = hyper_search.best_estimator_.named_steps['rfr'].feature_importances_
#         indices = np.argsort(importances)
#         plt.title('Feature Importances')
#         plt.barh(range(len(indices)), importances[indices], color='b', align='center')
#         plt.yticks(range(len(indices)), [features[i] for i in indices])
#         plt.xlabel('Relative Importance')
#         plt.show()
        
        prediction = model.predict(X_test)
        
        if 'Цена' in X_test.columns or 'Цена' in X_train.columns: print('\n=======ALERT DATA LEAK!======\n')
        
        return pd.Series(prediction, index=X_test.index, name='Цена')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_train_rent, df_train_rent_target, random_state=0, test_size=0.3)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
pred_train_rent_rfr = rent_model_03(X_train, X_test, y_train)
print(pred_train_rent_rfr.shape)
pred_train_rent_rfr

In [ ]:
print(f"score = {rmsle(y_test, pred_train_rent_rfr)}") # score = 0.35763077798677656, 10 деревьев, 
# score = 0.35028523871246453 на 100

In [ ]:
pred_rfr_knn = rent_model_03(X_train2, X_test2, y_train2)

In [ ]:
print(f"score = {rmsle(np.expm1(y_test2), np.expm1(pred_rfr_knn))}") #

In [ ]:
# def rfr_model(X, y):
# # Perform Grid-Search
#     gsc = GridSearchCV(
#         estimator=RandomForestRegressor(),
#         param_grid={
#             'max_depth': range(3,7),
#             'n_estimators': (10, 50, 100, 1000),
#         },
#         cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
    
#     grid_result = gsc.fit(X, y)
#     best_params = grid_result.best_params_
    
#     rfr = RandomForestRegressor(max_depth=best_params["max_depth"], n_estimators=best_params["n_estimators"],                               random_state=False, verbose=False)
# # Perform K-Fold CV
#    scores = cross_val_score(rfr, X, y, cv=10, scoring='neg_mean_absolute_error')

#     return scores

# scores = cross_val_score(rfr, X, y, cv=10, scoring='neg_mean_absolute_error')
# predictions = cross_val_predict(rfr, X, y, cv=10)

## Sale

In [ ]:
def sale_model_01(X_train, X_test, y_train):
        class SaleKNNImputer(BaseEstimator, TransformerMixin):
            def fit(self, X, y):
                return self
            def transform(self, X):
                cols = [
                    'Количество комнат', 
                    'Общая площадь', 
                    'Жилая площадь', 
                    'Площадь кухни',
                    'Этаж', 
                    'Этажность', 
                    'Тип квартиры', 
                    'Планировка', 
                    'Ремонт', 
                    'Санузел',
                    'Количество балконов', 
                    'Количество лоджий', 
                    'Интернет', 
                    'Торг',
                    'Чистая продажа', 
                    'Ипотека', 
                    'Возможен обмен', 
                    'Внесен задаток',
                    'Находится в залоге', 
                    'Форма собственности', 
                    'Год сдачи', 
                    'Тип дома',
                    'Материал дома', 
                    'Количество фотографий', 
                    'lat_h', 
                    'lon_h', 
                    'dist_center', 
                    'dist_center2'
                ]
                
                return X[cols].copy()

        pipeline = Pipeline([
            ('imputer', SaleKNNImputer()),
            ('scaler', StandardScaler()),
            ('knn', KNeighborsRegressor()),
        ])

        param_grid = {
            'knn__n_neighbors': [5],
            'knn__p': [1]
        }
        
        kfold = KFold(n_splits=3, shuffle=True, random_state=0)
        hyper_search = GridSearchCV(pipeline, param_grid, scoring=rmse, cv=kfold, n_jobs=5, refit=True, verbose=50)
        hyper_search.fit(X_train, y_train)
        model = hyper_search.best_estimator_
        print(f"RentKNNModel: best_params = {hyper_search.best_params_}, best_score = {hyper_search.best_score_}")
        y_pred = model.predict(X_test)
        
        return y_pred

In [ ]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(X2, y2, random_state=0, test_size=0.5)
X_train3.shape, X_test3.shape, y_train3.shape, y_test3.shape

In [ ]:
y_train3 = np.log1p(y_train3)
y_test3 = np.log1p(y_test3)

In [ ]:
# pred_knn_sale = sale_model_01(X_train3, X_test3, y_train3)
# pred_knn_sale;

In [ ]:
print(f"score = {rmsle(np.expm1(y_test3), np.expm1(pred_knn_sale))}") #score = 0.2634184160979688

In [ ]:
pred_knn_sale.shape

In [ ]:
def sale_model_02(X_train, X_test, y_train):
        class SaleLRImputer(BaseEstimator, TransformerMixin):
            def fit(self, X, y):
                return self

            def transform(self, X):
                cols = [
                    'Количество комнат', 
                    'Общая площадь', 
                    'Жилая площадь', 
                    'Площадь кухни',
                    'Этаж', 
                    'Этажность', 
                    'Тип квартиры', 
                    'Планировка', 
                    'Ремонт', 
                    'Санузел',
                    'Количество балконов', 
                    'Количество лоджий', 
                    'Интернет', 
                    'Торг',
                    'Чистая продажа', 
                    'Ипотека', 
                    'Возможен обмен', 
                    'Внесен задаток',
                    'Находится в залоге', 
                    'Форма собственности', 
                    'Год сдачи', 
                    'Тип дома',
                    'Материал дома', 
                    'Количество фотографий', 
                    'lat_h', 
                    'lon_h', 
                    'dist_center', 
                    'dist_center2',
                    'dist_center3',
                    'dist_center4',
                    'index_price_perv',
                    'index_price_vtor',
                    'inds'
                ]
                return X[cols].copy()

        pipeline = Pipeline([
            ('imputer', SaleLRImputer()),
            ('scaler', MinMaxScaler()),
            ('lr', Ridge()),
        ])

        param_grid = {
            'lr__alpha': [1, 10, 100, 1000],
        }
        
        hyper_search = GridSearchCV(pipeline, param_grid, scoring=rmse, cv=3, n_jobs=5, refit=True, verbose=2)
        hyper_search.fit(X_train, y_train)
        model = hyper_search.best_estimator_
        print(f"RentLRModel:best_params = {hyper_search.best_params_}, best_score = {hyper_search.best_score_}")
        y_pred = model.predict(X_test)
        return y_pred

In [ ]:
X_test_knn3 = X_test3.copy()
X_test_knn3 = pd.concat([X_test_knn3, pd.Series(pred_knn_sale, index=X_test3.index, name='knn')], axis=1, sort=False)

In [ ]:
X_test_knn3['knn'] = np.log1p(X_test_knn3['knn'])
y_test_knn3 = y_test3.copy()
#y_test_knn = np.log1p(y_test)

In [ ]:
X_train4, X_test4, y_train4, y_test4 = train_test_split(X_test_knn3, y_test_knn3, random_state=0, test_size=0.3)
X_train4.shape, X_test4.shape, y_train4.shape, y_test4.shape

In [ ]:
pred_lr_sale = sale_model_02(X_train4, X_test4, y_train4)
pred_lr_sale.shape

In [ ]:
print(f"score = {rmsle(np.expm1(y_test4), np.expm1(pred_lr_sale))}") #score = 0.3327222930861954

In [ ]:
def sale_model_03(X_train, X_test, y_train):
        class SaleRFRImputer(BaseEstimator, TransformerMixin):
            def fit(self, X, y):
                return self
            def transform(self, X):
                cols = [
                    'Количество комнат', 
                    'Общая площадь', 
                    'Жилая площадь', 
                    'Площадь кухни',
                    'Этаж', 
                    'Этажность', 
                    'Тип квартиры', 
                    'Планировка', 
                    'Ремонт', 
                    'Санузел',
                    'Количество балконов', 
                    'Количество лоджий', 
                    'Интернет', 
                    'Торг',
                    'Чистая продажа', 
                    'Ипотека', 
                    'Возможен обмен', 
                    'Внесен задаток',
                    'Находится в залоге', 
                    'Форма собственности', 
                    'Год сдачи', 
                    'Тип дома',
                    'Материал дома', 
                    'Количество фотографий', 
                    'lat_h', 
                    'lon_h', 
                    'dist_center', 
                    'dist_center2',
                    #'dist_center3',
                    #'dist_center4',
                    'index_price_perv',
                    'index_price_vtor',
                    'inds',
                    #'zp',
                    #'zp2',
                    #'metro',
                    'house_age',
                    #'not_first_not_top',
                    'not_first_floor',
                    'not_top_floor',
                    'has_balcony',
                    'region',
                    'town',
                    'urb'
                ]
                return X[cols].copy()

        pipe = Pipeline([
            ('imputer', SaleRFRImputer()),
            ('rfr', RandomForestRegressor()),
        ])

        param_grid={
            'rfr__max_depth': [25],
            'rfr__n_estimators': [10],
            #'rfr__max_features': ['sqrt'],
            #'rfr__min_samples_split': range(2, 10),
            #'rfr__min_samples_leaf': range(1, 500, 50),
            #'rfr__warm_start': [False, True],
            #'rfr__bootstrap': [False],
        },
        
        kfold = KFold(n_splits=3, shuffle=True, random_state=0)
        hyper_search = GridSearchCV(pipe, param_grid, scoring=rmsle_score, cv=kfold, n_jobs=2, refit=True, verbose=50)
        hyper_search.fit(X_train, y_train)
        model = hyper_search.best_estimator_
        print(f"SaleRFRModel: best_params = {hyper_search.best_params_}, best_score = {hyper_search.best_score_}")
        
#         # посмотрим на важность признаков
#         features = X_train.columns
#         importances = hyper_search.best_estimator_.named_steps['rfr'].feature_importances_
#         indices = np.argsort(importances)
#         plt.title('Feature Importances')
#         plt.barh(range(len(indices)), importances[indices], color='b', align='center')
#         plt.yticks(range(len(indices)), [features[i] for i in indices])
#         plt.xlabel('Relative Importance')
#         plt.show()
        
        prediction = model.predict(X_test)
                
        if 'Цена' in X_test.columns or 'Цена' in X_train.columns: print('\n=======ALERT DATA LEAK!======\n')
        
        return pd.Series(prediction, index=X_test.index, name='Цена')

In [ ]:
X_train5, X_test5, y_train5, y_test5 = train_test_split(df_train_sale, df_train_sale_target, random_state=0, test_size=0.3)
X_train5.shape, X_test5.shape, y_train5.shape, y_test5.shape

In [ ]:
pred_train_sale_rfr = sale_model_03(X_train5, X_test5, y_train5)
print(pred_train_sale_rfr.shape)
pred_train_sale_rfr

In [ ]:
print(f"score = {rmsle(y_test5, pred_train_sale_rfr)}") 

In [ ]:
pred_rfr_sale_knn = sale_model_03(X_train4, X_test4, y_train4)

In [ ]:
print(f"score = {rmsle(np.expm1(y_test4), np.expm1(pred_rfr_sale_knn))}") #score = 0.3582030741056765

## Test

## Test Data Preprocessing

In [ ]:
def clean_test_data(df_input):
    df_output = df_input.copy()

    df_output['Форма собственности'] = df_output['Форма собственности'].replace(
        {'другое (укажите в описании)': 'другое'}) 
    df_output['Материал дома'] = df_output['Материал дома'].replace({'кирпич-монолит':'кирпич - монолит'})
    
    return df_output

In [ ]:
df_test_preproc_NA_OUT_ENC = df_test.pipe(fill_nan).pipe(clean_test_data).pipe(feature_encoding)
df_test_preproc_NA_OUT_ENC.shape
del df_test

## Fit Rent

In [ ]:
df_test_rent = \
df_test_preproc_NA_OUT_ENC[df_test_preproc_NA_OUT_ENC['Рубрика'] == 'Аренда'].pipe(add_features)
df_test_rent.drop(cols_to_drop_rent, axis=1, inplace=True)
df_test_rent.shape

In [ ]:
#pred_test_rent_rfr = rent_model_03(df_train_rent, df_test_rent, df_train_rent_target)

In [ ]:
model = RandomForestRegressor(n_estimators=100, verbose=2, n_jobs=2)
model.fit(df_train_rent, df_train_rent_target)

In [ ]:
pred_test_rent_rfr = model.predict(df_test_rent)

In [ ]:
pred_test_rent_rfr.shape # best_score = -0.35762453844675574

In [ ]:
pred_test_rent_rfr = pd.Series(pred_test_rent_rfr, index=df_test_rent.index, name='Цена')
pred_test_rent_rfr

## Fit Sale

In [ ]:
df_test_sale = df_test_preproc_NA_OUT_ENC[df_test_preproc_NA_OUT_ENC['Рубрика'] == 'Продажа']
df_test_sale.shape

In [ ]:
del df_test_preproc_NA_OUT_ENC

In [ ]:
df_test_sale = df_test_sale.pipe(add_features)
df_test_sale.drop(cols_to_drop_sale, axis=1, inplace=True)
df_test_sale.shape

In [ ]:
#pred_test_sale_rfr = sale_model_03(df_train_sale, df_test_sale, df_train_sale_target)

In [ ]:
model = RandomForestRegressor(n_estimators=100, verbose=2, n_jobs=2)
model.fit(df_train_sale, df_train_sale_target)
pred_test_sale_rfr = model.predict(df_test_sale)

In [ ]:
pred_test_sale_rfr.shape # best_score = -0.1689683180613796

In [ ]:
pred_test_sale_rfr = pd.Series(pred_test_sale_rfr, index=df_test_sale.index, name='Цена')
pred_test_sale_rfr

In [ ]:
del model

## Combining predictions

In [ ]:
print(pred_test_rent_rfr.shape)
pred_test_rent_rfr

In [ ]:
pred_rent = pd.Series(pred_test_rent_rfr, index=df_test_rent.index, name='Цена')
pred_rent.shape
pred_rent

In [ ]:
print(pred_test_sale_rfr.shape)
pred_test_sale_rfr

In [ ]:
pred_sale = pd.Series(pred_test_sale_rfr, index=df_sale_test.index, name='Цена');
pred_sale.shape
pred_sale

In [ ]:
pred = pd.concat([pred_rent, pred_sale], axis=0).sort_index()
pred.shape
pred

In [ ]:
pd.DataFrame(zip(pred.index.values, pred.values), columns=['ID', 'Цена']).to_csv('pred_01_comp.csv', sep=',', index=False)

## Memory Monitoring

In [ ]:
# какие десять переменных занимают больше всего памяти
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

In [ ]:
%reset